# Tutorial 2. Sessions and State

:::{note}
:icon: false

#### How Panel sessions work, and how to use `pn.state` safely

This tutorial explains the runtime model behind Panel apps:

- what a **session** is
- where app **state** lives
- when to use `panel serve` vs `pn.serve`
- how to use `pn.state.add_periodic_callback`, `pn.state.execute`, and `pn.state.schedule_task`

## Learning goals

After this tutorial, you should be able to:

1. Explain how a browser tab maps to a server-side session.
2. Describe the role of the Bokeh `Document` in a Panel app.
3. Choose between `panel serve` and `pn.serve` for a deployment scenario.
4. Use `pn.state` APIs for periodic, immediate, and scheduled background work.
:::

---

In [ ]:
import panel as pn

pn.extension()

## 1) Mental model: session-backed applications

Panel apps are server applications. Your Python process owns the app state, while the browser renders synchronized frontend models.

```{mermaid}
flowchart LR
  subgraph S["Server Side Panel Session"]
    direction TB
    T["Template + Panel objects<br/>(layouts / widgets / panes)"]
    D["Bokeh Document<br/>(contains Bokeh models)"]
    H["HTML page"]
    W["WebSocket<br/>(Bokeh protocol)"]

    T -->|servable| D
    D --> H
  end

  B["Browser"]

  H --> B
  D <--> W
  W <--> B
```

Key idea:

- each browser tab gets a **server session**
- each session has its own **Bokeh `Document`**
- user interactions flow over WebSocket and trigger Python callbacks

---

## 2) Session lifecycle (request, sync, reconnect)

The sequence below captures the typical lifecycle.

```{mermaid}
sequenceDiagram
  autonumber
  participant U as User Browser (tab)
  participant H as HTTP Endpoint
  participant WS as WebSocket Endpoint
  participant APP as Bokeh Application (per app path)
  participant SES as ServerSession (per session id)
  participant DOC as Document (models + callbacks)
  participant PN as Panel App Code (build/attach)

  U->>H: GET /app
  H-->>U: HTML + Bokeh autoload bootstrap (ws url + session info)

  U->>WS: Open WebSocket (Bokeh protocol)
  WS->>SES: Create or resume ServerSession (session id)
  SES->>APP: Request document for session
  APP->>DOC: Create Document
  APP->>PN: Run app init (script or function) for this session
  PN->>DOC: Add root models (template/layout) + register callbacks
  DOC-->>U: Send initial document/models

  loop Live interaction
    U->>WS: UI events / widget changes
    WS->>DOC: Apply patches to models
    DOC->>PN: Invoke Python callbacks
    PN->>DOC: Mutate models/state
    DOC-->>U: Send patches / updates
  end

  U-->>WS: WebSocket closes (tab close / reload / network drop)
  WS-->>SES: Mark session disconnected (kept alive briefly)
  alt Reconnect within timeout
    U->>WS: Reconnect with same session id
    WS->>SES: Reuse existing ServerSession + Document
    SES-->>U: Resume syncing
  else Expire / server restart
    SES-->>DOC: Close session, drop document, stop callbacks
  end
```

## 3) Multi-user behavior: same code, separate sessions

One app process can serve many users at once. Session-local state is isolated by default.

    
```{mermaid}
flowchart LR
  classDef server fill:#F5F7FA,stroke:#6B7A90,stroke-width:1px;
  classDef browser fill:#EEF3FF,stroke:#6B7A90,stroke-width:1px;

  subgraph S["Panel Server (Python)"]
    direction TB
    A["Panel App Code"]
    D1["Session: Bokeh Document<br/>+ pn.state<br/>+ callbacks"]
    D2["Session: Bokeh Document<br/>+ pn.state<br/>+ callbacks"]
    D3["Session: Bokeh Document<br/>+ pn.state<br/>+ callbacks"]
    A --> D1
    A --> D2
    A --> D3
  end
  class S server

  subgraph B1["Browser"]
    direction TB
    F1["BokehJS / PanelJS<br/>(frontend models + rendering)"]
  end
  class B1 browser

  subgraph B2["Browser"]
    direction TB
    F2["BokehJS / PanelJS<br/>(frontend models + rendering)"]
  end
  class B2 browser

  subgraph B3["Browser"]
    direction TB
    F3["BokehJS / PanelJS<br/>(frontend models + rendering)"]
  end
  class B3 browser

  D1 -. "WebSocket sync" .- F1
  D2 -. "WebSocket sync" .- F2
  D3 -. "WebSocket sync" .- F3
```

## 4) `panel serve` vs `pn.serve`

Both run Panel apps, but they fit different workflows.

### `panel serve` (CLI-first)

Use the command line when you want a deployable, ops-friendly server process.

```bash
panel serve app.py --autoreload --port 5006
```

Best for:

- production-like serving and process management
- serving one or more scripts/files by path
- explicit server flags and lifecycle control from CLI

### `pn.serve` (Python-first)

Use inside Python when you need to start a server programmatically.

```python
import panel as pn

pn.extension()

app = pn.Column("# Hello", pn.widgets.IntSlider(name="Value"))

pn.serve(app, port=5006, show=False)
```

Best for:

- tests, demos, and scripted startup
- dynamic app construction in Python
- embedding server startup in an existing Python workflow

### Rule of thumb

- choose `panel serve` for deployment and operational clarity
- choose `pn.serve` when your Python code should control server creation


## 5) What `pn.state` is (and is not)

`pn.state` is Panel's runtime state interface. It gives you access to session/runtime context and scheduling helpers.

```{admonition} Important
:class: important
`pn.state` itself is process-global as an API surface, but many values exposed through it are session-aware when used inside callbacks.
```

Common uses:

- inspect request/session context (for authenticated apps, query args, etc.)
- register periodic/background tasks
- offload work without blocking UI responsiveness

Avoid:

- treating module-level mutable globals as user state
- running long blocking work directly in widget callbacks



## 6) Scheduling methods

In addition to various session and user state, the `pn.state` object provides a number of methods

- `add_periodic_callback`: Schedule a repeating task
- `execute`: Run a function on the event loop (or on a thread)
- `on_session_created`: Schedule a function to run before a session is created
- `on_session_destroyed`: Schedule a function to run before a session is removed
- `onload`: Schedule a function to run once the session has been fully initialized
- `schedule_task`: Scheduling tasks that run independently of a session


### `pn.state.add_periodic_callback`

Use this to run a function repeatedly on an interval.

In [ ]:
import datetime as dt

pn.extension()

clock = pn.pane.Str("")

def tick():
    clock.object = f"Server time: {dt.datetime.now():%H:%M:%S}"

cb = pn.state.add_periodic_callback(tick, period=1000, start=True)

pn.Column("## Periodic callback", clock)

In [ ]:
cb.stop()

:::{note} Notes

- `period` is milliseconds
- keep callback bodies short
- stop callbacks when no longer needed (for long-lived sessions)

:::

### `pn.state.execute`

Use `execute` to schedule work on Panel's event loop without blocking immediate UI handling.

In [ ]:
import time

status = pn.pane.Str("Idle")

def do_work():
    # Simulate heavier work.
    time.sleep(1)
    status.object = "Done"

def on_click(_):
    status.object = "Working..."
    pn.state.execute(do_work)

button = pn.widgets.Button(name="Run")
button.on_click(on_click)

pn.Column(button, status)

Use this pattern when callbacks should remain responsive while deferring extra work.

## 7) Common pitfalls

1. **Blocking callbacks**: long CPU/I/O work can freeze interactivity.
2. **Global mutable state leaks**: users accidentally influence each other.
3. **Forgotten periodic callbacks**: tasks continue running longer than expected.

## Recap

You now have the core runtime model:

- Panel apps are session-backed server apps.
- Each user session gets its own document/callback graph.
- `panel serve` is best for deployment; `pn.serve` is best for programmatic control.
- `pn.state` gives practical hooks for periodic, immediate, and scheduled work.

With this model, session bugs become much easier to reason about and debug.